# User Overview Analysis

In [1]:
import os
import sys
# Add the 'scripts' directory to the Python path for module imports
sys.path.append(os.path.abspath(os.path.join('..', 'scripts')))

In [ ]:
# import load_data from scripts module
from db_connect import load_data

xdr_df = load_data()